# Game Record Setup

TODO: Need to rethink all steps for proper formulas and calculations.

## Table of Contents

* [Data Setup](#data_setup)  
* [Model Setup](#model_setup)  
* [Game Records Table Creation](#game_records)
    * [Win Percentage](#win_pct)
    * [Expected Value](#ev)
    * [Game Score](#game_score)
    * [Recommended Bet Amount](#rec_bet)
* [Save to Database](#save)

## Imports and Global Settings

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from scipy import stats
from tensorflow import keras
from sqlalchemy import create_engine
from pycaret.regression import *
from pycaret.classification import *

# Pandas Settings
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 1000)
pd.options.display.max_info_columns = 200
pd.options.display.precision = 5

<a id=data_setup></a>
## Database Connection

In [2]:
username = 'postgres'
password = ''
endpoint = ''
database = 'nba_betting'
port = '5432'

connection = create_engine(f'postgresql+psycopg2://{username}:{password}@{endpoint}/{database}').connect()

## Load Data

TODO: Ability to input date or dates to be loaded.

In [3]:
query = """
SELECT cnc.game_id,
 cnc.game_date,
 cnc.home_team,
 cnc.away_team,
 cnc.home_score,
 cnc.away_score,
 cnc.home_result,
 cnc.covers_game_url,
 cnc.league_year,
 cnc.home_spread,
 cnc.home_spread_result,
 cnc.pred_date,
 cnc.fd_line_home,
 cnc.fd_line_price_home,
 cnc.fd_line_away,
 cnc.fd_line_price_away,
 cnc.dk_line_home,
 cnc.dk_line_price_home,
 cnc.dk_line_away,
 cnc.dk_line_price_away,
 cnc.covers_consenses_home,
 cnc.covers_consenses_away,
 mr.*
FROM combined_nba_covers AS cnc 
LEFT OUTER JOIN nba_model_ready AS mr 
ON cnc.game_id = mr.game_id"""

In [4]:
df = pd.read_sql(sql=query, con=connection)

In [5]:
df = df.loc[:,~df.columns.duplicated()].copy()

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9200 entries, 0 to 9199
Columns: 1236 entries, game_id to day_of_season
dtypes: bool(1), datetime64[ns](2), float64(1212), int64(4), object(17)
memory usage: 86.7+ MB


In [7]:
df.head(10)

,game_id,game_date,home_team,away_team,home_score,away_score,home_result,covers_game_url,league_year,home_spread,home_spread_result,pred_date,fd_line_home,fd_line_price_home,fd_line_away,fd_line_price_away,dk_line_home,dk_line_price_home,dk_line_away,dk_line_price_away,covers_consenses_home,covers_consenses_away,CLS_TARGET_home_margin_GT_home_spread,REG_TARGET_actual_home_margin,home_team_num,away_team_num,league_year_end,gp,win,loss,w_pct,mins,pts,fgm,fga,fg_pct,fg3m,fg3a,fg3_pct,ftm,fta,ft_pct,oreb,dreb,reb,ast,tov,stl,blk,blka,pf,pfd,p_m,gp_rank,gp_vla,gp_vla_std,win_rank,win_vla,win_vla_std,loss_rank,loss_vla,loss_vla_std,w_pct_rank,w_pct_vla,w_pct_vla_std,mins_rank,mins_vla,mins_vla_std,pts_rank,pts_vla,pts_vla_std,fgm_rank,fgm_vla,fgm_vla_std,fga_rank,fga_vla,fga_vla_std,fg_pct_rank,fg_pct_vla,fg_pct_vla_std,fg3m_rank,fg3m_vla,fg3m_vla_std,fg3a_rank,fg3a_vla,fg3a_vla_std,fg3_pct_rank,fg3_pct_vla,fg3_pct_vla_std,ftm_rank,ftm_vla,ftm_vla_std,fta_rank,fta_vla,fta_vla_std,ft_pct_rank,ft_pct_vla,ft_pct_vla_std,oreb_rank,oreb_vla,oreb_vla_std,dreb_rank,dreb_vla,dreb_vla_std,reb_rank,reb_vla,reb_vla_std,ast_rank,ast_vla,ast_vla_std,tov_rank,tov_vla,tov_vla_std,stl_rank,stl_vla,stl_vla_std,blk_rank,blk_vla,blk_vla_std,blka_rank,blka_vla,blka_vla_std,pf_rank,pf_vla,pf_vla_std,pfd_rank,pfd_vla,pfd_vla_std,p_m_rank,p_m_vla,p_m_vla_std,gp_opp,win_opp,loss_opp,w_pct_opp,mins_opp,pts_opp,fgm_opp,fga_opp,fg_pct_opp,fg3m_opp,fg3a_opp,fg3_pct_opp,ftm_opp,fta_opp,ft_pct_opp,oreb_opp,dreb_opp,reb_opp,ast_opp,tov_opp,stl_opp,blk_opp,blka_opp,pf_opp,pfd_opp,p_m_opp,gp_rank_opp,gp_vla_opp,gp_vla_std_opp,win_rank_opp,win_vla_opp,win_vla_std_opp,loss_rank_opp,loss_vla_opp,loss_vla_std_opp,w_pct_rank_opp,w_pct_vla_opp,w_pct_vla_std_opp,mins_rank_opp,mins_vla_opp,mins_vla_std_opp,pts_rank_opp,pts_vla_opp,pts_vla_std_opp,fgm_rank_opp,fgm_vla_opp,fgm_vla_std_opp,fga_rank_opp,fga_vla_opp,fga_vla_std_opp,fg_pct_rank_opp,fg_pct_vla_opp,fg_pct_vla_std_opp,fg3m_rank_opp,fg3m_vla_opp,fg3m_vla_std_opp,fg3a_rank_opp,fg3a_vla_opp,fg3a_vla_std_opp,fg3_pct_rank_opp,fg3_pct_vla_opp,fg3_pct_vla_std_opp,ftm_rank_opp,ftm_vla_opp,ftm_vla_std_opp,fta_rank_opp,fta_vla_opp,fta_vla_std_opp,ft_pct_rank_opp,ft_pct_vla_opp,ft_pct_vla_std_opp,oreb_rank_opp,oreb_vla_opp,oreb_vla_std_opp,dreb_rank_opp,dreb_vla_opp,dreb_vla_std_opp,reb_rank_opp,reb_vla_opp,reb_vla_std_opp,ast_rank_opp,ast_vla_opp,ast_vla_std_opp,tov_rank_opp,tov_vla_opp,tov_vla_std_opp,stl_rank_opp,stl_vla_opp,stl_vla_std_opp,blk_rank_opp,blk_vla_opp,blk_vla_std_opp,blka_rank_opp,blka_vla_opp,blka_vla_std_opp,pf_rank_opp,pf_vla_opp,pf_vla_std_opp,pfd_rank_opp,pfd_vla_opp,pfd_vla_std_opp,p_m_rank_opp,p_m_vla_opp,p_m_vla_std_opp,offrtg,defrtg,netrtg,ast_pct,ast_v_tov,ast_ratio,oreb_pct,dreb_pct,reb_pct,tov_pct,efg_pct,ts_pct,pace,pie,poss,...,opp_fgm_c3,opp_fga_c3,opp_fg_pct_c3,opp_fgm_atb3,opp_fga_atb3,opp_fg_pct_atb3,opp_fgm_ra_rank,opp_fgm_ra_vla,opp_fgm_ra_vla_std,opp_fga_ra_rank,opp_fga_ra_vla,opp_fga_ra_vla_std,opp_fg_pct_ra_rank,opp_fg_pct_ra_vla,opp_fg_pct_ra_vla_std,opp_fgm_paint_rank,opp_fgm_paint_vla,opp_fgm_paint_vla_std,opp_fga_paint_rank,opp_fga_paint_vla,opp_fga_paint_vla_std,opp_fg_pct_paint_rank,opp_fg_pct_paint_vla,opp_fg_pct_paint_vla_std,opp_fgm_mr_rank,opp_fgm_mr_vla,opp_fgm_mr_vla_std,opp_fga_mr_rank,opp_fga_mr_vla,opp_fga_mr_vla_std,opp_fg_pct_mr_rank,opp_fg_pct_mr_vla,opp_fg_pct_mr_vla_std,opp_fgm_lc3_rank,opp_fgm_lc3_vla,opp_fgm_lc3_vla_std,opp_fga_lc3_rank,opp_fga_lc3_vla,opp_fga_lc3_vla_std,opp_fg_pct_lc3_rank,opp_fg_pct_lc3_vla,opp_fg_pct_lc3_vla_std,opp_fgm_rc3_rank,opp_fgm_rc3_vla,opp_fgm_rc3_vla_std,opp_fga_rc3_rank,opp_fga_rc3_vla,opp_fga_rc3_vla_std,opp_fg_pct_rc3_rank,opp_fg_pct_rc3_vla,opp_fg_pct_rc3_vla_std,opp_fgm_c3_rank,opp_fgm_c3_vla,opp_fgm_c3_vla_std,opp_fga_c3_rank,opp_fga_c3_vla,opp_fga_c3_vla_std,opp_fg_pct_c3_rank,opp_fg_pct_c3_vla,opp_fg_pct_c3_vla_std,opp_fgm_atb3_rank,opp_fgm_atb3_vla,opp_fgm_atb3_vla_std,opp_fga_atb3_rank,opp_fga_atb3_vla,opp_fga_atb3_vla_std,o

<a id=model_setup></a>
## Load Models

### AutoML

In [8]:
AutoML_model_CLS = load_model('../models/AutoML/Baseline_LR_CLS_PyCaret')
AutoML_model_REG = load_model('../models/AutoML/Baseline_Lasso_Reg_PyCaret')

Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded


### DL

In [9]:
DL_model_CLS = keras.models.load_model('../models/Deep_Learning/NBA_Stats_CLS_Baseline_2022_09_06_20_08_19')
DL_model_REG = keras.models.load_model('../models/Deep_Learning/NBA_Stats_REG_Baseline_2022_09_06_20_00_26')

## Model Predictions

In [10]:
main_features = ['home_team_num', 'away_team_num', 'league_year_end', 'home_spread']
nba_stats_features_vla = [feature for feature in list(df) if feature[-3:] == 'vla']

ml_pred_feature_set = main_features + nba_stats_features_vla
dl_pred_feature_set = main_features + nba_stats_features_vla

### AutoML

In [11]:
automl_predictions_CLS = predict_model(AutoML_model_CLS, data=df[ml_pred_feature_set])

In [12]:
automl_predictions_CLS

,home_team_num,away_team_num,league_year_end,home_spread,gp_vla,win_vla,loss_vla,w_pct_vla,mins_vla,pts_vla,fgm_vla,fga_vla,fg_pct_vla,fg3m_vla,fg3a_vla,fg3_pct_vla,ftm_vla,fta_vla,ft_pct_vla,oreb_vla,dreb_vla,reb_vla,ast_vla,tov_vla,stl_vla,blk_vla,blka_vla,pf_vla,pfd_vla,p_m_vla,offrtg_vla,defrtg_vla,netrtg_vla,ast_pct_vla,ast_v_tov_vla,ast_ratio_vla,oreb_pct_vla,dreb_pct_vla,reb_pct_vla,tov_pct_vla,efg_pct_vla,ts_pct_vla,pace_vla,pie_vla,poss_vla,fta_rate_vla,opp_efg_pct_vla,opp_fta_rate_vla,opp_tov_pct_vla,opp_oreb_pct_vla,pts_off_tov_vla,second_pts_vla,fbps_vla,pitp_vla,opp_pts_off_tov_vla,opp_second_pts_vla,opp_fbps_vla,opp_pitp_vla,pct_fga_2pt_vla,pct_fga_3pt_vla,pct_pts_2pt_vla,pct_pts_2pt_mid_vla,pct_pts_3pt_vla,pct_pts_fbps_vla,pct_pts_ft_vla,pct_pts_off_tov_vla,pct_pts_pitp_vla,pct_ast_2fgm_vla,pct_uast_2fgm_vla,pct_ast_3fgm_vla,pct_uast_3fgm_vla,pct_ast_fgm_vla,pct_uast_fgm_vla,opp_fgm_vla,opp_fga_vla,opp_fg_pct_vla,opp_3pm_vla,opp_3pa_vla,opp_3pt_pct_vla,opp_ftm_vla,opp_fta_vla,opp_ft_pct_vla,opp_oreb_vla,opp_dreb_vla,opp_reb_vla,opp_ast_vla,opp_tov_vla,opp_stl_vla,opp_blk_vla,opp_blka_vla,opp_pf_vla,opp_pfd_vla,opp_pts_vla,opp_pm_vla,dist_feet_vla,dist_miles_vla,dist_miles_off_vla,dist_miles_def_vla,avg_speed_vla,avg_speed_off_vla,avg_speed_def_vla,fgm_ra_vla,fga_ra_vla,fg_pct_ra_vla,fgm_paint_vla,fga_paint_vla,fg_pct_paint_vla,fgm_mr_vla,fga_mr_vla,fg_pct_mr_vla,fgm_lc3_vla,fga_lc3_vla,fg_pct_lc3_vla,fgm_rc3_vla,fga_rc3_vla,fg_pct_rc3_vla,fgm_c3_vla,fga_c3_vla,fg_pct_c3_vla,fgm_atb3_vla,fga_atb3_vla,fg_pct_atb3_vla,opp_fgm_ra_vla,opp_fga_ra_vla,opp_fg_pct_ra_vla,opp_fgm_paint_vla,opp_fga_paint_vla,opp_fg_pct_paint_vla,opp_fgm_mr_vla,opp_fga_mr_vla,opp_fg_pct_mr_vla,opp_fgm_lc3_vla,opp_fga_lc3_vla,opp_fg_pct_lc3_vla,opp_fgm_rc3_vla,opp_fga_rc3_vla,opp_fg_pct_rc3_vla,opp_fgm_c3_vla,opp_fga_c3_vla,opp_fg_pct_c3_vla,opp_fgm_atb3_vla,opp_fga_atb3_vla,opp_fg_pct_atb3_vla,screen_ast_vla,screen_ast_pts_vla,deflections_vla,off_loose_ball_rec_vla,def_loose_ball_rec_vla,loose_ball_rec_vla,pct_loose_ball_rec_off_vla,pct_loose_ball_rec_def_vla,charges_drawn_vla,contested_2pt_vla,contested_3pt_vla,contested_shots_vla,Label,Score
0,20,28,22,1.0,0.00000,-4.50000,4.50000,-0.05603,-0.06000,-4.02000,-2.98667,-1.86667,-2.38333,0.78667,1.74667,0.38000,1.16333,2.43000,-3.19667,1.17667,0.47333,1.65667,-2.91000,-0.56333,-0.63667,0.18000,-0.02333,0.75667,0.75333,-0.29667,-1.60667,-0.99000,-0.61333,-2.80667,-0.14567,-1.61667,1.76667,1.46333,1.20667,-0.26000,-1.84667,-1.58000,-2.31967,-0.99333,-181.13333,0.03343,-0.94000,0.01253,-0.75667,-1.46333,-0.56333,1.21667,-1.98667,-5.72000,-0.95667,-1.07333,-1.18000,-4.01667,-2.85667,2.85667,-5.06000,-1.25000,3.39333,-1.37333,1.67000,-0.01333,-3.81000,-5.03000,5.03000,-2.85667,2.85667,-2.80667,2.80667,-2.08333,-2.16000,-0.01273,0.27333,1.75000,-0.01030,0.16667,0.53000,-0.01127,-1.02667,0.17333,-0.85667,-0.50333,-1.06333,-0.72333,-0.02333,0.18000,0.75333,0.75667,-3.72333,0.29667,299.87833,0.05567,0.02867,0.02800,-0.00167,-0.06467,0.06100,-1.18667,-0.56333,-0.03493,-1.72000,-2.33667,-0.04667,-0.78333,-0.69333,-0.04453,0.12667,0.07667,0.02017,-0.24667,-0.50333,-0.01043,-0.13333,-0.32333,0.00590,0.84333,2.12667,0.00237,-0.49333,0.03333,-0.01830,-1.52000,-3.54000,-0.00073,-0.37667,-0.37667,-0.01950,0.02000,0.37333,-0.02583,-0.16000,0.08667,-0.04687,-0.14667,0.38667,-0.03460,0.45667,1.32333,-0.00060,0.19600,1.27167,-1.15167,0.34800,0.16067,0.49867,0.01553,-0.01553,-0.13867,-1.91100,1.27167,-0.64100,False,0.5219
1,6,17,22,-8.5,0.00000,2.50000,-2.50000,0.03097,-0.16000,-3.02000,-0.98667,-3.56667,0.71667,-0.91333,-2.45333,-0.02000,-0.13667,0.23000,-1.39667,-0.12333,-0.22667,-0.34333,0.39000,0.63667,-0.53667,-0.52000,-0.02333,-2.24333,0.35333,1.90333,-0.50667,-2.39000,1.88667,2.59333,-0.06567,0.68333,0.26667,-0.23667,0.60667,1.04000,0.45333,0.42000,-2.00967,1.60667,-183.13333,0.01343,-1.14000,-0.02747,0.04333,0.23667,-0.46333,-0.68333,-1.48667,0.98000,1.14333,-0.77333,-0.28000,-0.61667,1.24333,-1.24333,1.14

In [13]:
automl_predictions_REG = predict_model(AutoML_model_REG, data=df[ml_pred_feature_set])

In [14]:
automl_predictions_REG

,home_team_num,away_team_num,league_year_end,home_spread,gp_vla,win_vla,loss_vla,w_pct_vla,mins_vla,pts_vla,fgm_vla,fga_vla,fg_pct_vla,fg3m_vla,fg3a_vla,fg3_pct_vla,ftm_vla,fta_vla,ft_pct_vla,oreb_vla,dreb_vla,reb_vla,ast_vla,tov_vla,stl_vla,blk_vla,blka_vla,pf_vla,pfd_vla,p_m_vla,offrtg_vla,defrtg_vla,netrtg_vla,ast_pct_vla,ast_v_tov_vla,ast_ratio_vla,oreb_pct_vla,dreb_pct_vla,reb_pct_vla,tov_pct_vla,efg_pct_vla,ts_pct_vla,pace_vla,pie_vla,poss_vla,fta_rate_vla,opp_efg_pct_vla,opp_fta_rate_vla,opp_tov_pct_vla,opp_oreb_pct_vla,pts_off_tov_vla,second_pts_vla,fbps_vla,pitp_vla,opp_pts_off_tov_vla,opp_second_pts_vla,opp_fbps_vla,opp_pitp_vla,pct_fga_2pt_vla,pct_fga_3pt_vla,pct_pts_2pt_vla,pct_pts_2pt_mid_vla,pct_pts_3pt_vla,pct_pts_fbps_vla,pct_pts_ft_vla,pct_pts_off_tov_vla,pct_pts_pitp_vla,pct_ast_2fgm_vla,pct_uast_2fgm_vla,pct_ast_3fgm_vla,pct_uast_3fgm_vla,pct_ast_fgm_vla,pct_uast_fgm_vla,opp_fgm_vla,opp_fga_vla,opp_fg_pct_vla,opp_3pm_vla,opp_3pa_vla,opp_3pt_pct_vla,opp_ftm_vla,opp_fta_vla,opp_ft_pct_vla,opp_oreb_vla,opp_dreb_vla,opp_reb_vla,opp_ast_vla,opp_tov_vla,opp_stl_vla,opp_blk_vla,opp_blka_vla,opp_pf_vla,opp_pfd_vla,opp_pts_vla,opp_pm_vla,dist_feet_vla,dist_miles_vla,dist_miles_off_vla,dist_miles_def_vla,avg_speed_vla,avg_speed_off_vla,avg_speed_def_vla,fgm_ra_vla,fga_ra_vla,fg_pct_ra_vla,fgm_paint_vla,fga_paint_vla,fg_pct_paint_vla,fgm_mr_vla,fga_mr_vla,fg_pct_mr_vla,fgm_lc3_vla,fga_lc3_vla,fg_pct_lc3_vla,fgm_rc3_vla,fga_rc3_vla,fg_pct_rc3_vla,fgm_c3_vla,fga_c3_vla,fg_pct_c3_vla,fgm_atb3_vla,fga_atb3_vla,fg_pct_atb3_vla,opp_fgm_ra_vla,opp_fga_ra_vla,opp_fg_pct_ra_vla,opp_fgm_paint_vla,opp_fga_paint_vla,opp_fg_pct_paint_vla,opp_fgm_mr_vla,opp_fga_mr_vla,opp_fg_pct_mr_vla,opp_fgm_lc3_vla,opp_fga_lc3_vla,opp_fg_pct_lc3_vla,opp_fgm_rc3_vla,opp_fga_rc3_vla,opp_fg_pct_rc3_vla,opp_fgm_c3_vla,opp_fga_c3_vla,opp_fg_pct_c3_vla,opp_fgm_atb3_vla,opp_fga_atb3_vla,opp_fg_pct_atb3_vla,screen_ast_vla,screen_ast_pts_vla,deflections_vla,off_loose_ball_rec_vla,def_loose_ball_rec_vla,loose_ball_rec_vla,pct_loose_ball_rec_off_vla,pct_loose_ball_rec_def_vla,charges_drawn_vla,contested_2pt_vla,contested_3pt_vla,contested_shots_vla,Label
0,20,28,22,1.0,0.00000,-4.50000,4.50000,-0.05603,-0.06000,-4.02000,-2.98667,-1.86667,-2.38333,0.78667,1.74667,0.38000,1.16333,2.43000,-3.19667,1.17667,0.47333,1.65667,-2.91000,-0.56333,-0.63667,0.18000,-0.02333,0.75667,0.75333,-0.29667,-1.60667,-0.99000,-0.61333,-2.80667,-0.14567,-1.61667,1.76667,1.46333,1.20667,-0.26000,-1.84667,-1.58000,-2.31967,-0.99333,-181.13333,0.03343,-0.94000,0.01253,-0.75667,-1.46333,-0.56333,1.21667,-1.98667,-5.72000,-0.95667,-1.07333,-1.18000,-4.01667,-2.85667,2.85667,-5.06000,-1.25000,3.39333,-1.37333,1.67000,-0.01333,-3.81000,-5.03000,5.03000,-2.85667,2.85667,-2.80667,2.80667,-2.08333,-2.16000,-0.01273,0.27333,1.75000,-0.01030,0.16667,0.53000,-0.01127,-1.02667,0.17333,-0.85667,-0.50333,-1.06333,-0.72333,-0.02333,0.18000,0.75333,0.75667,-3.72333,0.29667,299.87833,0.05567,0.02867,0.02800,-0.00167,-0.06467,0.06100,-1.18667,-0.56333,-0.03493,-1.72000,-2.33667,-0.04667,-0.78333,-0.69333,-0.04453,0.12667,0.07667,0.02017,-0.24667,-0.50333,-0.01043,-0.13333,-0.32333,0.00590,0.84333,2.12667,0.00237,-0.49333,0.03333,-0.01830,-1.52000,-3.54000,-0.00073,-0.37667,-0.37667,-0.01950,0.02000,0.37333,-0.02583,-0.16000,0.08667,-0.04687,-0.14667,0.38667,-0.03460,0.45667,1.32333,-0.00060,0.19600,1.27167,-1.15167,0.34800,0.16067,0.49867,0.01553,-0.01553,-0.13867,-1.91100,1.27167,-0.64100,0
1,6,17,22,-8.5,0.00000,2.50000,-2.50000,0.03097,-0.16000,-3.02000,-0.98667,-3.56667,0.71667,-0.91333,-2.45333,-0.02000,-0.13667,0.23000,-1.39667,-0.12333,-0.22667,-0.34333,0.39000,0.63667,-0.53667,-0.52000,-0.02333,-2.24333,0.35333,1.90333,-0.50667,-2.39000,1.88667,2.59333,-0.06567,0.68333,0.26667,-0.23667,0.60667,1.04000,0.45333,0.42000,-2.00967,1.60667,-183.13333,0.01343,-1.14000,-0.02747,0.04333,0.23667,-0.46333,-0.68333,-1.48667,0.98000,1.14333,-0.77333,-0.28000,-0.61667,1.24333,-1.24333,1.14000,-0.95000,-1.5

### DL

In [15]:
dl_predictions_CLS = DL_model_CLS.predict(df[dl_pred_feature_set])

In [16]:
dl_predictions_CLS

array([[ 0.],
       [ 1.],
       [ 0.],
       ...,
       [nan],
       [nan],
       [nan]], dtype=float32)

In [17]:
dl_predictions_CLS = dl_predictions_CLS.flatten()

In [18]:
dl_predictions_CLS

array([ 0.,  1.,  0., ..., nan, nan, nan], dtype=float32)

In [19]:
dl_predictions_REG = DL_model_REG.predict(df[dl_pred_feature_set]).flatten()

In [20]:
dl_predictions_REG

array([ -2809.3384 ,    255.82689, -11568.143  , ...,          nan,
                nan,          nan], dtype=float32)

<a id=game_records></a>
## Game Records Table

In [21]:
game_records = pd.DataFrame({'line_hv': 0 - df['home_spread'],
                             'ml_reg_prediction': automl_predictions_REG['Label'],
                             'dl_reg_prediction': dl_predictions_REG,
                             'ml_cls_prediction': automl_predictions_CLS['Label'],
                             'ml_cls_pred_proba': automl_predictions_CLS['Score'],
                             'dl_cls_prediction': dl_predictions_CLS})

### Which Team to Bet On - Home or Away

In [22]:
game_records['ml_cls_pred_direction'] = game_records['ml_cls_prediction'].apply(lambda x: 'Home' if x == 'True' else 'Away')
game_records['dl_cls_pred_direction'] = game_records['dl_cls_prediction'].apply(lambda x: 'Home' if x == 1.0 else 'Away')

### Line and Line Price
TODO: Bring in actual line prices. Lower Priority

In [23]:
game_records['home_line'] = df['home_spread']
game_records['home_line_price'] = -110
game_records['away_line'] = -df['home_spread']
game_records['away_line_price'] = -110

### Recommended Bet Direction
TODO: Create logic to choose between competing ML and DL recommendations. Lower Priority

In [24]:
game_records["rec_bet_direction"] = game_records.apply(
            lambda x: x["ml_cls_pred_direction"] if x["ml_cls_pred_direction"] == x[
                "dl_cls_pred_direction"] else "Warning-Models Differ",
            axis=1,
        )

<a id=game_score></a>
### Game Score
TODO  
Combines ML and DL predictions, win%, and expected value with other outside factors to create a single final metric on a scale of 1 to 100.

In [25]:
game_records['game_score'] = game_records['ml_cls_pred_proba']

<a id=rec_bet></a>
### Recommended Bet Amount
TODO: Combines game scores with bankroll management plan to create bet amount on each game, each day.

In [26]:
available_funds_query = """SELECT balance
                           FROM bank_account
                           WHERE datetime = (SELECT MAX(datetime)
                           FROM bank_account);"""

In [27]:
current_available_funds = pd.read_sql(sql=available_funds_query,
                                      con=connection)["balance"]

In [28]:
unit_size = 0.01
unit_amount = current_available_funds * unit_size

In [29]:
def calc_amount_to_bet(x, unit_size):
    if x["game_score"] > 95:
        units_to_bet = 5
    elif x["game_score"] > 90:
        units_to_bet = 3
    elif x["game_score"] > 85:
        units_to_bet = 1
    else:
        units_to_bet = 0
    amount_to_bet = unit_size * units_to_bet
    return amount_to_bet

In [30]:
game_records["rec_bet_amount"] = game_records.apply(
    calc_amount_to_bet, unit_size=unit_amount, axis=1)

In [31]:
game_records.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9200 entries, 0 to 9199
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   line_hv                9200 non-null   float64
 1   ml_reg_prediction      9200 non-null   int64  
 2   dl_reg_prediction      6901 non-null   float32
 3   ml_cls_prediction      9200 non-null   object 
 4   ml_cls_pred_proba      9200 non-null   float64
 5   dl_cls_prediction      6901 non-null   float32
 6   ml_cls_pred_direction  9200 non-null   object 
 7   dl_cls_pred_direction  9200 non-null   object 
 8   home_line              9200 non-null   float64
 9   home_line_price        9200 non-null   int64  
 10  away_line              9200 non-null   float64
 11  away_line_price        9200 non-null   int64  
 12  rec_bet_direction      9200 non-null   object 
 13  game_score             9200 non-null   float64
 14  rec_bet_amount         9200 non-null   float64
dtypes: f

### Additional Game Info

In [32]:
game_records['game_id'] = df['game_id']
game_records['game_info'] = df['covers_game_url']
game_records['date'] = df['game_date'].dt.date
game_records['home'] = df['home_team']
game_records['away'] = df['away_team']
game_records["game_result"] = df.apply(lambda x: x["home_score"] - x["away_score"] if x["home_score"] else "Pending",
            axis=1)

### Game Record Cleanup

In [33]:
ordered_cols = ['game_id',
 'game_info',
 'date',
 'home',
 'away',
 'home_line',
 'home_line_price',
 'away_line',
 'away_line_price',
 'ml_cls_prediction',
 'ml_cls_pred_proba',               
 'ml_cls_pred_direction',
 'ml_reg_prediction',               
 'dl_cls_prediction',
 'dl_cls_pred_direction',
 'dl_reg_prediction',
 'game_score',
 'rec_bet_direction',
 'rec_bet_amount',
 'game_result']

In [34]:
game_records = game_records[ordered_cols]

In [35]:
game_records

,game_id,game_info,date,home,away,home_line,home_line_price,away_line,away_line_price,ml_cls_prediction,ml_cls_pred_proba,ml_cls_pred_direction,ml_reg_prediction,dl_cls_prediction,dl_cls_pred_direction,dl_reg_prediction,game_score,rec_bet_direction,rec_bet_amount,game_result
0,20220410NYKTOR,/sport/basketball/nba/boxscore/251783,2022-04-10,NYK,TOR,1.0,-110,-1.0,-110,False,0.5219,Away,0,0.0,Away,-2809.33838,0.5219,Away,0.0,11.0
1,20220410CLEMIL,/sport/basketball/nba/boxscore/251775,2022-04-10,CLE,MIL,-8.5,-110,8.5,-110,False,0.5470,Away,8,1.0,Home,255.82689,0.5470,Warning-Models Differ,0.0,18.0
2,20220408DETMIL,/sport/basketball/nba/boxscore/251756,2022-04-08,DET,MIL,10.5,-110,-10.5,-110,False,0.5029,Away,-9,0.0,Away,-11568.14258,0.5029,Away,0.0,-30.0
3,20220403CLEPHI,/sport/basketball/nba/boxscore/251723,2022-04-03,CLE,PHI,5.5,-110,-5.5,-110,True,0.5144,Home,-4,1.0,Home,369.18335,0.5144,Home,0.0,-4.0
4,20220406DETDAL,/sport/basketball/nba/boxscore/251743,2022-04-06,DET,DAL,8.5,-110,-8.5,-110,False,0.5136,Away,-7,0.0,Away,-11880.92090,0.5136,Away,0.0,-18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9195,20141126SASIND,/sport/basketball/nba/boxscore/32428,2014-11-26,SAS,IND,-13.0,-110,13.0,-110,True,0.5145,Home,12,NaN,Away,NaN,0.5145,Warning-Models Differ,0.0,6.0
9196,20141122SASBKN,/sport/basketball/nba/boxscore/32150,2014-11-22,SAS,BKN,-10.0,-110,10.0,-110,False,0.5274,Away,9,NaN,Away,NaN,0.5274,Away,0.0,12.0
9197,20141117SASPHI,/sport/basketball/nba/boxscore/30907,2014-11-17,SAS,PHI,-16.5,-110,16.5,-110,True,0.5496,Home,15,NaN,Away,NaN,0.5496,Warning-Models Differ,0.0,25.0
9198,20141108SASNOP,/sport/basketball/nba/boxscore/29837,2014-11-08,SAS,NOP,-7.5,-110,7.5,-110,True,0.6648,Home,7,NaN,Away,NaN,0.6648,Warning-Models Differ,0.0,-1.0


In [36]:
game_records.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9200 entries, 0 to 9199
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   game_id                9200 non-null   object 
 1   game_info              9200 non-null   object 
 2   date                   9200 non-null   object 
 3   home                   9200 non-null   object 
 4   away                   9200 non-null   object 
 5   home_line              9200 non-null   float64
 6   home_line_price        9200 non-null   int64  
 7   away_line              9200 non-null   float64
 8   away_line_price        9200 non-null   int64  
 9   ml_cls_prediction      9200 non-null   object 
 10  ml_cls_pred_proba      9200 non-null   float64
 11  ml_cls_pred_direction  9200 non-null   object 
 12  ml_reg_prediction      9200 non-null   int64  
 13  dl_cls_prediction      6901 non-null   float32
 14  dl_cls_pred_direction  9200 non-null   object 
 15  dl_r

<a id=save></a>
## Save to RDS

In [37]:
# game_records.to_sql(
#             name="game_records",
#             con=connection,
#             index=False,
#             if_exists="append",
#         )